# Merged train+val and merged info and y

# Add Marks z-score stat

In [1]:

# === 輔助函式 ===
import numpy as np
from scipy.stats import kurtosis, skew
from numpy.fft import fft


# ============================================================
# ⚙️ 通用 Normalization Function
# ============================================================
def normalize_signal(vals, global_mean=None, global_std=None, mode="none"):
    """
    多模式 normalization 支援：
      - "none": 不做任何處理
      - "zscore": (x - μ) / σ
      - "log_zscore": ((log1p(x)) - μ_log) / σ_log
      - "log_only": 只取 log1p，不標準化
    """
    if mode == "none":
        return vals

    elif mode == "zscore":
        if global_mean is None or global_std is None:
            raise ValueError("Missing global_mean/global_std for zscore normalization.")
        return (vals - global_mean) / (global_std + 1e-8)

    elif mode == "log_zscore":
        if global_mean is None or global_std is None:
            raise ValueError("Missing global_mean/global_std for log_zscore normalization.")
        vals = np.log1p(np.clip(vals, a_min=0, a_max=None))
        return (vals - global_mean) / (global_std + 1e-8)

    elif mode == "log_only":
        return np.log1p(np.clip(vals, a_min=0, a_max=None))

    else:
        raise ValueError(f"Unknown normalization mode: {mode}")


# ============================================================
# 🧬 region_zsignal (with normalization selection)
# ============================================================
def region_zsignal(bw, chrom, start, end, global_mean=None, global_std=None,
                   mark_name=None, cell_name=None, norm_mode="none"):
    """
    Extracts region-level features with multiple normalization modes.
    """
    feature_keys = [
        "mean", "std", "min", "max", "diff",
        "gradient_mean", "slope", "kurtosis", "skewness",
        "entropy", "autocorr", "laplacian"
    ]

    chroms = bw.chroms()
    if chrom not in chroms:
        return {k: 0.0 for k in feature_keys}

    chrom_length = chroms[chrom]
    start, end = max(0, int(start)), min(int(end), chrom_length)
    if end <= start:
        return {k: 0.0 for k in feature_keys}

    vals = np.array(bw.values(chrom, start, end, numpy=True))
    vals = vals[~np.isnan(vals)]
    if len(vals) == 0:
        return {k: 0.0 for k in feature_keys}

    # === 🔧 Apply normalization (可動態切換) ===
    vals_norm = normalize_signal(vals, global_mean, global_std, mode=norm_mode)

    # === Feature Extraction ===
    local_mean = np.mean(vals_norm)
    local_std  = np.std(vals_norm)
    local_min  = np.min(vals_norm)
    local_max  = np.max(vals_norm)
    local_diff = local_max - local_min

    # Gradient / slope
    if len(vals_norm) > 1:
        diffs = np.diff(vals_norm)
        gradient_mean = np.mean(np.abs(diffs))
        try:
            slope = np.polyfit(np.arange(len(vals_norm)), vals_norm, 1)[0]
        except Exception:
            slope = 0.0
    else:
        gradient_mean, slope = 0.0, 0.0

    # Shape-based descriptors
    sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
    asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0

    # Entropy
    p = np.abs(vals_norm)
    p_sum = np.sum(p)
    local_entropy = -np.sum((p / (p_sum + 1e-8)) * np.log2(p / (p_sum + 1e-8))) if p_sum > 0 else 0.0

    # Autocorrelation & Laplacian
    autocorr = np.corrcoef(vals_norm[:-1], vals_norm[1:])[0, 1] if len(vals_norm) > 2 else 0.0
    laplacian = np.mean(np.abs(vals_norm[:-2] - 2 * vals_norm[1:-1] + vals_norm[2:])) if len(vals_norm) > 3 else 0.0

    result = {
        "mean": local_mean,
        "std": local_std,
        "min": local_min,
        "max": local_max,
        "diff": local_diff,
        "gradient_mean": gradient_mean,
        "slope": slope,
        "kurtosis": sharpness,
        "skewness": asymmetry,
        "entropy": local_entropy,
        "autocorr": autocorr,
        "laplacian": laplacian,
    }

    # 保險處理 nan/inf
    for k, v in result.items():
        if not np.isfinite(v):
            result[k] = 0.0

    return result


def get_tss_region(row, window=1000):
    """根據 strand (+/-) 給出 promoter (TSS ± window) 區域"""
    if row["strand"] == "+":
        start = max(0, row["TSS_start"] - window)
        end = row["TSS_end"] + window
    else:  # strand == "-"
        start = max(0, row["TSS_start"] - window)
        end = row["TSS_end"] + window
    return start, end

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import pyBigWig
from tqdm import tqdm

# === 基本設定 ===
base = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/ML4G_Project_1_Data"
output_dir = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/reference/0. data/bigwig250/"
stats_path = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/global_norm_stats.json"

marks = ["DNase", "H3K27ac", "H3K4me3", "H3K27me3", "H3K36me3", "H3K4me1", "H3K9me3"]
cells = ["X1","X2","X3"]

tss_window = 250  # +/- 5kb
ref_path = "../preprocessed_data/reference/reference_gene_table.tsv"

# === normalization 模式可選 ===
# 可選 "none", "zscore", "log_zscore", "log_only"
norm_mode = "log_zscore"

# ============================================================
# 載入 reference & global stats
# ============================================================
ref_genes = pd.read_csv(ref_path, sep="\t")
print(f"📖 Loaded reference: {len(ref_genes)} genes")

with open(stats_path, "r") as f:
    stats = json.load(f)




# ============================================================
# 主迴圈
# ============================================================
for cell in cells:
    print(f"\n🧬 Processing cell line: {cell}")
    genes = ref_genes.copy()

    for mark in marks:
        bw_path = os.path.join(base, f"{mark}-bigwig", f"{cell}.bw")
        if not os.path.exists(bw_path):
            bw_path = os.path.join(base, f"{mark}-bigwig", f"{cell}.bigwig")
        if not os.path.exists(bw_path):
            print(f"⚠️ Missing {mark} ({cell})")
            continue

        print(f"\n📂 Reading {mark} ({cell}) ...")
        bw = pyBigWig.open(bw_path)
        chroms = bw.chroms()

        key = f"{mark}_{cell}"
        if key not in stats:
            print(f"⚠️ No global stat found for {key}")
            continue

        # --- 根據 norm_mode 選取 global mean/std ---
        if norm_mode == "log_zscore":
            global_mean = stats[key]["log1p"]["mean"]
            global_std = stats[key]["log1p"]["std"]
        elif norm_mode == "zscore":
            global_mean = stats[key]["linear"]["mean"]
            global_std = stats[key]["linear"]["std"]
        else:
            global_mean = global_std = None

        # === 動態命名 prefix ===
        prefix = {
            "none": "raw",
            "zscore": "z",
            "log_zscore": "logz",
            "log_only": "log"
        }[norm_mode]

        # === 建立空字典儲存所有 feature ===
        base_keys = [
            "mean", "std", "min", "max", "diff",
            "gradient_mean", "slope", "kurtosis", "skewness",
            "entropy", "autocorr", "laplacian"
        ]


        # --- 對 mean/std/min/max/diff 加上 prefix ---
        prefixed_keys = [f"{prefix}_{k}" for k in base_keys[:5]]
        other_keys = base_keys[5:]
        feat_keys = prefixed_keys + other_keys

        # === 建立空字典儲存 features ===
        gene_features = {f"{mark}_gene_{k}": [] for k in feat_keys}
        tss_features  = {f"{mark}_tss_{k}": [] for k in feat_keys}


        # === 主迴圈 ===
        for _, row in tqdm(genes.iterrows(), total=len(genes)):
            chrom = row["chr"]
            if chrom not in chroms:
                for d in (gene_features, tss_features):
                    for k in d.keys():
                        d[k].append(np.nan)
                continue

            g_stats = region_zsignal(bw, chrom, row["gene_start"], row["gene_end"],
                                    global_mean, global_std, mark, cell, norm_mode)

            for k, v in g_stats.items():
                target_key = f"{mark}_gene_{prefix}_{k}" if k in base_keys[:5] else f"{mark}_gene_{k}"
                gene_features[target_key].append(v)

            # TSS
            tss_start, tss_end = get_tss_region(row, window=tss_window)
            t_stats = region_zsignal(bw, chrom, tss_start, tss_end,
                                    global_mean, global_std, mark, cell, norm_mode)

            for k, v in t_stats.items():
                target_key = f"{mark}_tss_{prefix}_{k}" if k in base_keys[:5] else f"{mark}_tss_{k}"
                tss_features[target_key].append(v)


        bw.close()

        # === 合併 ===
        for feat_dict in (gene_features, tss_features):
            for col, vals in feat_dict.items():
                genes[col] = vals

    # === 儲存結果 ===
    out_suffix = {
        "none": "raw",
        "zscore": "zscore",
        "log_zscore": "logzscore",
        "log_only": "log"
    }[norm_mode]

    out_path = os.path.join(output_dir, f"{cell}_{out_suffix}.tsv")
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    genes.to_csv(out_path, sep="\t", index=False)

    print(f"\n✅ Saved {out_suffix} features for {cell} → {out_path}")


📖 Loaded reference: 18268 genes

🧬 Processing cell line: X1

📂 Reading DNase (X1) ...


  0%|          | 57/18268 [00:00<03:07, 97.21it/s] /var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_74255/4018982790.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_74255/4018982790.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
 44%|████▍     | 8129/18268 [01:06<01:24, 119.63it/s]

In [31]:
import os
import json
import numpy as np
import pandas as pd
import pyBigWig
from tqdm import tqdm

# === 基本設定 ===
base = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/ML4G_Project_1_Data"
output_dir = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/reference/0. data/bigwig500_one_side/"
stats_path = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/global_norm_stats.json"

marks = ["DNase", "H3K27ac", "H3K4me3", "H3K27me3", "H3K36me3", "H3K4me1", "H3K9me3"]
cells = ["X1","X2","X3"]

tss_window = 500  # +/- 5kb
ref_path = "../preprocessed_data/reference/reference_gene_table.tsv"

# === normalization 模式可選 ===
# 可選 "none", "zscore", "log_zscore", "log_only"
norm_mode = "log_zscore"

# ============================================================
# 載入 reference & global stats
# ============================================================
ref_genes = pd.read_csv(ref_path, sep="\t")
print(f"📖 Loaded reference: {len(ref_genes)} genes")

with open(stats_path, "r") as f:
    stats = json.load(f)




# ============================================================
# 主迴圈
# ============================================================
for cell in cells:
    print(f"\n🧬 Processing cell line: {cell}")
    genes = ref_genes.copy()

    for mark in marks:
        bw_path = os.path.join(base, f"{mark}-bigwig", f"{cell}.bw")
        if not os.path.exists(bw_path):
            bw_path = os.path.join(base, f"{mark}-bigwig", f"{cell}.bigwig")
        if not os.path.exists(bw_path):
            print(f"⚠️ Missing {mark} ({cell})")
            continue

        print(f"\n📂 Reading {mark} ({cell}) ...")
        bw = pyBigWig.open(bw_path)
        chroms = bw.chroms()

        key = f"{mark}_{cell}"
        if key not in stats:
            print(f"⚠️ No global stat found for {key}")
            continue

        # --- 根據 norm_mode 選取 global mean/std ---
        if norm_mode == "log_zscore":
            global_mean = stats[key]["log1p"]["mean"]
            global_std = stats[key]["log1p"]["std"]
        elif norm_mode == "zscore":
            global_mean = stats[key]["linear"]["mean"]
            global_std = stats[key]["linear"]["std"]
        else:
            global_mean = global_std = None

        # === 動態命名 prefix ===
        prefix = {
            "none": "raw",
            "zscore": "z",
            "log_zscore": "logz",
            "log_only": "log"
        }[norm_mode]

        # === 建立空字典儲存所有 feature ===
        base_keys = [
            "mean", "std", "min", "max", "diff",
            "gradient_mean", "slope", "kurtosis", "skewness",
            "entropy", "autocorr", "laplacian"
        ]


        # --- 對 mean/std/min/max/diff 加上 prefix ---
        prefixed_keys = [f"{prefix}_{k}" for k in base_keys[:5]]
        other_keys = base_keys[5:]
        feat_keys = prefixed_keys + other_keys

        # === 建立空字典儲存 features ===
        gene_features = {f"{mark}_gene_{k}": [] for k in feat_keys}
        tss_features  = {f"{mark}_tss_{k}": [] for k in feat_keys}


        # === 主迴圈 ===
        for _, row in tqdm(genes.iterrows(), total=len(genes)):
            chrom = row["chr"]
            if chrom not in chroms:
                for d in (gene_features, tss_features):
                    for k in d.keys():
                        d[k].append(np.nan)
                continue

            g_stats = region_zsignal(bw, chrom, row["gene_start"], row["gene_end"],
                                    global_mean, global_std, mark, cell, norm_mode)

            for k, v in g_stats.items():
                target_key = f"{mark}_gene_{prefix}_{k}" if k in base_keys[:5] else f"{mark}_gene_{k}"
                gene_features[target_key].append(v)

            # TSS
            tss_start, tss_end = get_tss_region(row, window=tss_window)
            t_stats = region_zsignal(bw, chrom, tss_start, tss_end,
                                    global_mean, global_std, mark, cell, norm_mode)

            for k, v in t_stats.items():
                target_key = f"{mark}_tss_{prefix}_{k}" if k in base_keys[:5] else f"{mark}_tss_{k}"
                tss_features[target_key].append(v)


        bw.close()

        # === 合併 ===
        for feat_dict in (gene_features, tss_features):
            for col, vals in feat_dict.items():
                genes[col] = vals

    # === 儲存結果 ===
    out_suffix = {
        "none": "raw",
        "zscore": "zscore",
        "log_zscore": "logzscore",
        "log_only": "log"
    }[norm_mode]

    out_path = os.path.join(output_dir, f"{cell}_{out_suffix}.tsv")
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    genes.to_csv(out_path, sep="\t", index=False)

    print(f"\n✅ Saved {out_suffix} features for {cell} → {out_path}")


📖 Loaded reference: 18268 genes

🧬 Processing cell line: X1

📂 Reading DNase (X1) ...


  0%|          | 12/18268 [00:00<02:34, 118.07it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
 19%|█▊        | 3409/18268 [00:20<01:07, 219.61it/s]/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_ba


📂 Reading H3K27ac (X1) ...


  0%|          | 51/18268 [00:00<02:10, 139.39it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
 37%|███▋      | 6806/18268 [00:44<01:56, 98.18it/s] /opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_ba


📂 Reading H3K4me3 (X1) ...


...


📂 Reading H3K4me1 (X3) ...


  0%|          | 32/18268 [00:00<02:52, 105.82it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
 50%|████▉     | 9092/18268 [01:01<00:51, 178.62it/s]/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_ba


📂 Reading H3K9me3 (X3) ...


  0%|          | 49/18268 [00:00<02:32, 119.58it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
 45%|████▍     | 8216/18268 [00:55<00:56, 179.35it/s]/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_ba


✅ Saved logzscore features for X3 → /Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/reference/0. data/bigwig500_one_side/X3_logzscore.tsv


In [34]:
import os
import json
import numpy as np
import pandas as pd
import pyBigWig
from tqdm import tqdm

# === 基本設定 ===
base = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/ML4G_Project_1_Data"
output_dir = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/reference/0. data/bigwig5000_one_side/"
stats_path = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/global_norm_stats.json"

marks = ["DNase", "H3K27ac", "H3K4me3", "H3K27me3", "H3K36me3", "H3K4me1", "H3K9me3"]
cells = ["X1","X2","X3"]

tss_window = 5000  # +/- 5kb
ref_path = "../preprocessed_data/reference/reference_gene_table.tsv"

# === normalization 模式可選 ===
# 可選 "none", "zscore", "log_zscore", "log_only"
norm_mode = "log_zscore"

# ============================================================
# 載入 reference & global stats
# ============================================================
ref_genes = pd.read_csv(ref_path, sep="\t")
print(f"📖 Loaded reference: {len(ref_genes)} genes")

with open(stats_path, "r") as f:
    stats = json.load(f)




# ============================================================
# 主迴圈
# ============================================================
for cell in cells:
    print(f"\n🧬 Processing cell line: {cell}")
    genes = ref_genes.copy()

    for mark in marks:
        bw_path = os.path.join(base, f"{mark}-bigwig", f"{cell}.bw")
        if not os.path.exists(bw_path):
            bw_path = os.path.join(base, f"{mark}-bigwig", f"{cell}.bigwig")
        if not os.path.exists(bw_path):
            print(f"⚠️ Missing {mark} ({cell})")
            continue

        print(f"\n📂 Reading {mark} ({cell}) ...")
        bw = pyBigWig.open(bw_path)
        chroms = bw.chroms()

        key = f"{mark}_{cell}"
        if key not in stats:
            print(f"⚠️ No global stat found for {key}")
            continue

        # --- 根據 norm_mode 選取 global mean/std ---
        if norm_mode == "log_zscore":
            global_mean = stats[key]["log1p"]["mean"]
            global_std = stats[key]["log1p"]["std"]
        elif norm_mode == "zscore":
            global_mean = stats[key]["linear"]["mean"]
            global_std = stats[key]["linear"]["std"]
        else:
            global_mean = global_std = None

        # === 動態命名 prefix ===
        prefix = {
            "none": "raw",
            "zscore": "z",
            "log_zscore": "logz",
            "log_only": "log"
        }[norm_mode]

        # === 建立空字典儲存所有 feature ===
        base_keys = [
            "mean", "std", "min", "max", "diff",
            "gradient_mean", "slope", "kurtosis", "skewness",
            "entropy", "autocorr", "laplacian"
        ]


        # --- 對 mean/std/min/max/diff 加上 prefix ---
        prefixed_keys = [f"{prefix}_{k}" for k in base_keys[:5]]
        other_keys = base_keys[5:]
        feat_keys = prefixed_keys + other_keys

        # === 建立空字典儲存 features ===
        gene_features = {f"{mark}_gene_{k}": [] for k in feat_keys}
        tss_features  = {f"{mark}_tss_{k}": [] for k in feat_keys}


        # === 主迴圈 ===
        for _, row in tqdm(genes.iterrows(), total=len(genes)):
            chrom = row["chr"]
            if chrom not in chroms:
                for d in (gene_features, tss_features):
                    for k in d.keys():
                        d[k].append(np.nan)
                continue

            g_stats = region_zsignal(bw, chrom, row["gene_start"], row["gene_end"],
                                    global_mean, global_std, mark, cell, norm_mode)

            for k, v in g_stats.items():
                target_key = f"{mark}_gene_{prefix}_{k}" if k in base_keys[:5] else f"{mark}_gene_{k}"
                gene_features[target_key].append(v)

            # TSS
            tss_start, tss_end = get_tss_region(row, window=tss_window)
            t_stats = region_zsignal(bw, chrom, tss_start, tss_end,
                                    global_mean, global_std, mark, cell, norm_mode)

            for k, v in t_stats.items():
                target_key = f"{mark}_tss_{prefix}_{k}" if k in base_keys[:5] else f"{mark}_tss_{k}"
                tss_features[target_key].append(v)


        bw.close()

        # === 合併 ===
        for feat_dict in (gene_features, tss_features):
            for col, vals in feat_dict.items():
                genes[col] = vals

    # === 儲存結果 ===
    out_suffix = {
        "none": "raw",
        "zscore": "zscore",
        "log_zscore": "logzscore",
        "log_only": "log"
    }[norm_mode]

    out_path = os.path.join(output_dir, f"{cell}_{out_suffix}.tsv")
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    genes.to_csv(out_path, sep="\t", index=False)

    print(f"\n✅ Saved {out_suffix} features for {cell} → {out_path}")


📖 Loaded reference: 18268 genes

🧬 Processing cell line: X1

📂 Reading DNase (X1) ...


  3%|▎         | 596/18268 [00:04<02:29, 118.50it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
100%|██████████| 18268/18268 [02:01<00:00, 150.63it/s]



📂 Reading H3K27ac (X1) ...


  0%|          | 42/18268 [00:00<02:43, 111.16it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
100%|██████████| 18268/18268 [02:01<00:00, 150.17it/s]



📂 Reading H3K4me3 (X1) ...


...


📂 Reading H3K4me1 (X3) ...


  1%|          | 172/18268 [00:02<05:21, 56.27it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
100%|██████████| 18268/18268 [03:27<00:00, 87.96it/s] 
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/3769782981.py:126:


📂 Reading H3K9me3 (X3) ...


  1%|          | 167/18268 [00:01<02:30, 119.90it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
 95%|█████████▍| 17339/18268 [03:00<00:07, 123.90it/s]/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_


✅ Saved logzscore features for X3 → /Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/reference/0. data/bigwig5000_one_side/X3_logzscore.tsv


In [33]:
import os
import json
import numpy as np
import pandas as pd
import pyBigWig
from tqdm import tqdm

# === 基本設定 ===
base = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/ML4G_Project_1_Data"
output_dir = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/reference/0. data/bigwig3000_one_side/"
stats_path = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/global_norm_stats.json"

marks = ["DNase", "H3K27ac", "H3K4me3", "H3K27me3", "H3K36me3", "H3K4me1", "H3K9me3"]
cells = ["X1","X2","X3"]

tss_window = 3000  # +/- 5kb
ref_path = "../preprocessed_data/reference/reference_gene_table.tsv"

# === normalization 模式可選 ===
# 可選 "none", "zscore", "log_zscore", "log_only"
norm_mode = "log_zscore"

# ============================================================
# 載入 reference & global stats
# ============================================================
ref_genes = pd.read_csv(ref_path, sep="\t")
print(f"📖 Loaded reference: {len(ref_genes)} genes")

with open(stats_path, "r") as f:
    stats = json.load(f)




# ============================================================
# 主迴圈
# ============================================================
for cell in cells:
    print(f"\n🧬 Processing cell line: {cell}")
    genes = ref_genes.copy()

    for mark in marks:
        bw_path = os.path.join(base, f"{mark}-bigwig", f"{cell}.bw")
        if not os.path.exists(bw_path):
            bw_path = os.path.join(base, f"{mark}-bigwig", f"{cell}.bigwig")
        if not os.path.exists(bw_path):
            print(f"⚠️ Missing {mark} ({cell})")
            continue

        print(f"\n📂 Reading {mark} ({cell}) ...")
        bw = pyBigWig.open(bw_path)
        chroms = bw.chroms()

        key = f"{mark}_{cell}"
        if key not in stats:
            print(f"⚠️ No global stat found for {key}")
            continue

        # --- 根據 norm_mode 選取 global mean/std ---
        if norm_mode == "log_zscore":
            global_mean = stats[key]["log1p"]["mean"]
            global_std = stats[key]["log1p"]["std"]
        elif norm_mode == "zscore":
            global_mean = stats[key]["linear"]["mean"]
            global_std = stats[key]["linear"]["std"]
        else:
            global_mean = global_std = None

        # === 動態命名 prefix ===
        prefix = {
            "none": "raw",
            "zscore": "z",
            "log_zscore": "logz",
            "log_only": "log"
        }[norm_mode]

        # === 建立空字典儲存所有 feature ===
        base_keys = [
            "mean", "std", "min", "max", "diff",
            "gradient_mean", "slope", "kurtosis", "skewness",
            "entropy", "autocorr", "laplacian"
        ]


        # --- 對 mean/std/min/max/diff 加上 prefix ---
        prefixed_keys = [f"{prefix}_{k}" for k in base_keys[:5]]
        other_keys = base_keys[5:]
        feat_keys = prefixed_keys + other_keys

        # === 建立空字典儲存 features ===
        gene_features = {f"{mark}_gene_{k}": [] for k in feat_keys}
        tss_features  = {f"{mark}_tss_{k}": [] for k in feat_keys}


        # === 主迴圈 ===
        for _, row in tqdm(genes.iterrows(), total=len(genes)):
            chrom = row["chr"]
            if chrom not in chroms:
                for d in (gene_features, tss_features):
                    for k in d.keys():
                        d[k].append(np.nan)
                continue

            g_stats = region_zsignal(bw, chrom, row["gene_start"], row["gene_end"],
                                    global_mean, global_std, mark, cell, norm_mode)

            for k, v in g_stats.items():
                target_key = f"{mark}_gene_{prefix}_{k}" if k in base_keys[:5] else f"{mark}_gene_{k}"
                gene_features[target_key].append(v)

            # TSS
            tss_start, tss_end = get_tss_region(row, window=tss_window)
            t_stats = region_zsignal(bw, chrom, tss_start, tss_end,
                                    global_mean, global_std, mark, cell, norm_mode)

            for k, v in t_stats.items():
                target_key = f"{mark}_tss_{prefix}_{k}" if k in base_keys[:5] else f"{mark}_tss_{k}"
                tss_features[target_key].append(v)


        bw.close()

        # === 合併 ===
        for feat_dict in (gene_features, tss_features):
            for col, vals in feat_dict.items():
                genes[col] = vals

    # === 儲存結果 ===
    out_suffix = {
        "none": "raw",
        "zscore": "zscore",
        "log_zscore": "logzscore",
        "log_only": "log"
    }[norm_mode]

    out_path = os.path.join(output_dir, f"{cell}_{out_suffix}.tsv")
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    genes.to_csv(out_path, sep="\t", index=False)

    print(f"\n✅ Saved {out_suffix} features for {cell} → {out_path}")


📖 Loaded reference: 18268 genes

🧬 Processing cell line: X1

📂 Reading DNase (X1) ...


  2%|▏         | 369/18268 [00:02<02:38, 112.63it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
100%|██████████| 18268/18268 [01:55<00:00, 157.84it/s]



📂 Reading H3K27ac (X1) ...


  0%|          | 44/18268 [00:00<02:38, 114.68it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
100%|██████████| 18268/18268 [02:02<00:00, 149.23it/s]



📂 Reading H3K4me3 (X1) ...


...


📂 Reading H3K4me1 (X3) ...


  1%|          | 172/18268 [00:01<01:47, 169.02it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
100%|██████████| 18268/18268 [01:54<00:00, 160.04it/s]
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/4080024639.py:126


📂 Reading H3K9me3 (X3) ...


  0%|          | 46/18268 [00:00<02:43, 111.67it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
 95%|█████████▍| 17321/18268 [01:49<00:04, 216.11it/s]/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_b


✅ Saved logzscore features for X3 → /Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/reference/0. data/bigwig3000_one_side/X3_logzscore.tsv


In [32]:
import os
import json
import numpy as np
import pandas as pd
import pyBigWig
from tqdm import tqdm

# === 基本設定 ===
base = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/ML4G_Project_1_Data"
output_dir = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/reference/0. data/bigwig2000_one_side/"
stats_path = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/global_norm_stats.json"

marks = ["DNase", "H3K27ac", "H3K4me3", "H3K27me3", "H3K36me3", "H3K4me1", "H3K9me3"]
cells = ["X1","X2","X3"]

tss_window = 2000  # +/- 5kb
ref_path = "../preprocessed_data/reference/reference_gene_table.tsv"

# === normalization 模式可選 ===
# 可選 "none", "zscore", "log_zscore", "log_only"
norm_mode = "log_zscore"

# ============================================================
# 載入 reference & global stats
# ============================================================
ref_genes = pd.read_csv(ref_path, sep="\t")
print(f"📖 Loaded reference: {len(ref_genes)} genes")

with open(stats_path, "r") as f:
    stats = json.load(f)




# ============================================================
# 主迴圈
# ============================================================
for cell in cells:
    print(f"\n🧬 Processing cell line: {cell}")
    genes = ref_genes.copy()

    for mark in marks:
        bw_path = os.path.join(base, f"{mark}-bigwig", f"{cell}.bw")
        if not os.path.exists(bw_path):
            bw_path = os.path.join(base, f"{mark}-bigwig", f"{cell}.bigwig")
        if not os.path.exists(bw_path):
            print(f"⚠️ Missing {mark} ({cell})")
            continue

        print(f"\n📂 Reading {mark} ({cell}) ...")
        bw = pyBigWig.open(bw_path)
        chroms = bw.chroms()

        key = f"{mark}_{cell}"
        if key not in stats:
            print(f"⚠️ No global stat found for {key}")
            continue

        # --- 根據 norm_mode 選取 global mean/std ---
        if norm_mode == "log_zscore":
            global_mean = stats[key]["log1p"]["mean"]
            global_std = stats[key]["log1p"]["std"]
        elif norm_mode == "zscore":
            global_mean = stats[key]["linear"]["mean"]
            global_std = stats[key]["linear"]["std"]
        else:
            global_mean = global_std = None

        # === 動態命名 prefix ===
        prefix = {
            "none": "raw",
            "zscore": "z",
            "log_zscore": "logz",
            "log_only": "log"
        }[norm_mode]

        # === 建立空字典儲存所有 feature ===
        base_keys = [
            "mean", "std", "min", "max", "diff",
            "gradient_mean", "slope", "kurtosis", "skewness",
            "entropy", "autocorr", "laplacian"
        ]


        # --- 對 mean/std/min/max/diff 加上 prefix ---
        prefixed_keys = [f"{prefix}_{k}" for k in base_keys[:5]]
        other_keys = base_keys[5:]
        feat_keys = prefixed_keys + other_keys

        # === 建立空字典儲存 features ===
        gene_features = {f"{mark}_gene_{k}": [] for k in feat_keys}
        tss_features  = {f"{mark}_tss_{k}": [] for k in feat_keys}


        # === 主迴圈 ===
        for _, row in tqdm(genes.iterrows(), total=len(genes)):
            chrom = row["chr"]
            if chrom not in chroms:
                for d in (gene_features, tss_features):
                    for k in d.keys():
                        d[k].append(np.nan)
                continue

            g_stats = region_zsignal(bw, chrom, row["gene_start"], row["gene_end"],
                                    global_mean, global_std, mark, cell, norm_mode)

            for k, v in g_stats.items():
                target_key = f"{mark}_gene_{prefix}_{k}" if k in base_keys[:5] else f"{mark}_gene_{k}"
                gene_features[target_key].append(v)

            # TSS
            tss_start, tss_end = get_tss_region(row, window=tss_window)
            t_stats = region_zsignal(bw, chrom, tss_start, tss_end,
                                    global_mean, global_std, mark, cell, norm_mode)

            for k, v in t_stats.items():
                target_key = f"{mark}_tss_{prefix}_{k}" if k in base_keys[:5] else f"{mark}_tss_{k}"
                tss_features[target_key].append(v)


        bw.close()

        # === 合併 ===
        for feat_dict in (gene_features, tss_features):
            for col, vals in feat_dict.items():
                genes[col] = vals

    # === 儲存結果 ===
    out_suffix = {
        "none": "raw",
        "zscore": "zscore",
        "log_zscore": "logzscore",
        "log_only": "log"
    }[norm_mode]

    out_path = os.path.join(output_dir, f"{cell}_{out_suffix}.tsv")
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    genes.to_csv(out_path, sep="\t", index=False)

    print(f"\n✅ Saved {out_suffix} features for {cell} → {out_path}")


📖 Loaded reference: 18268 genes

🧬 Processing cell line: X1

📂 Reading DNase (X1) ...


  2%|▏         | 367/18268 [00:02<02:44, 108.89it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
100%|██████████| 18268/18268 [01:50<00:00, 165.80it/s]



📂 Reading H3K27ac (X1) ...


  0%|          | 49/18268 [00:00<02:06, 143.48it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
100%|██████████| 18268/18268 [01:58<00:00, 153.72it/s]



📂 Reading H3K4me3 (X1) ...


...


📂 Reading H3K4me1 (X3) ...


  1%|          | 165/18268 [00:01<01:38, 184.14it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
100%|██████████| 18268/18268 [01:58<00:00, 153.96it/s]
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/3065288594.py:126


📂 Reading H3K9me3 (X3) ...


  0%|          | 46/18268 [00:00<02:37, 115.59it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/2984093304.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
 95%|█████████▍| 17325/18268 [01:53<00:04, 196.04it/s]/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_b


✅ Saved logzscore features for X3 → /Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/reference/0. data/bigwig2000_one_side/X3_logzscore.tsv


In [26]:
import os
import json
import numpy as np
import pandas as pd
import pyBigWig
from tqdm import tqdm

# === 基本設定 ===
base = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/ML4G_Project_1_Data"
output_dir = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/reference/0. data/bigwig2000"
stats_path = "/Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/global_norm_stats.json"

marks = ["DNase", "H3K27ac", "H3K4me3", "H3K27me3", "H3K36me3", "H3K4me1", "H3K9me3"]
cells = ["X1","X2","X3"]

tss_window = 2000  # +/- 5kb
ref_path = "../preprocessed_data/reference/reference_gene_table.tsv"

# === normalization 模式可選 ===
# 可選 "none", "zscore", "log_zscore", "log_only"
norm_mode = "log_zscore"

# ============================================================
# 載入 reference & global stats
# ============================================================
ref_genes = pd.read_csv(ref_path, sep="\t")
print(f"📖 Loaded reference: {len(ref_genes)} genes")

with open(stats_path, "r") as f:
    stats = json.load(f)




# ============================================================
# 主迴圈
# ============================================================
for cell in cells:
    print(f"\n🧬 Processing cell line: {cell}")
    genes = ref_genes.copy()

    for mark in marks:
        bw_path = os.path.join(base, f"{mark}-bigwig", f"{cell}.bw")
        if not os.path.exists(bw_path):
            bw_path = os.path.join(base, f"{mark}-bigwig", f"{cell}.bigwig")
        if not os.path.exists(bw_path):
            print(f"⚠️ Missing {mark} ({cell})")
            continue

        print(f"\n📂 Reading {mark} ({cell}) ...")
        bw = pyBigWig.open(bw_path)
        chroms = bw.chroms()

        key = f"{mark}_{cell}"
        if key not in stats:
            print(f"⚠️ No global stat found for {key}")
            continue

        # --- 根據 norm_mode 選取 global mean/std ---
        if norm_mode == "log_zscore":
            global_mean = stats[key]["log1p"]["mean"]
            global_std = stats[key]["log1p"]["std"]
        elif norm_mode == "zscore":
            global_mean = stats[key]["linear"]["mean"]
            global_std = stats[key]["linear"]["std"]
        else:
            global_mean = global_std = None

        # === 動態命名 prefix ===
        prefix = {
            "none": "raw",
            "zscore": "z",
            "log_zscore": "logz",
            "log_only": "log"
        }[norm_mode]

        # === 建立空字典儲存所有 feature ===
        base_keys = [
            "mean", "std", "min", "max", "diff",
            "gradient_mean", "slope", "kurtosis", "skewness",
            "entropy", "autocorr", "laplacian"
        ]


        # --- 對 mean/std/min/max/diff 加上 prefix ---
        prefixed_keys = [f"{prefix}_{k}" for k in base_keys[:5]]
        other_keys = base_keys[5:]
        feat_keys = prefixed_keys + other_keys

        # === 建立空字典儲存 features ===
        gene_features = {f"{mark}_gene_{k}": [] for k in feat_keys}
        tss_features  = {f"{mark}_tss_{k}": [] for k in feat_keys}


        # === 主迴圈 ===
        for _, row in tqdm(genes.iterrows(), total=len(genes)):
            chrom = row["chr"]
            if chrom not in chroms:
                for d in (gene_features, tss_features):
                    for k in d.keys():
                        d[k].append(np.nan)
                continue

            g_stats = region_zsignal(bw, chrom, row["gene_start"], row["gene_end"],
                                    global_mean, global_std, mark, cell, norm_mode)

            for k, v in g_stats.items():
                target_key = f"{mark}_gene_{prefix}_{k}" if k in base_keys[:5] else f"{mark}_gene_{k}"
                gene_features[target_key].append(v)

            # TSS
            tss_start, tss_end = get_tss_region(row, window=tss_window)
            t_stats = region_zsignal(bw, chrom, tss_start, tss_end,
                                    global_mean, global_std, mark, cell, norm_mode)

            for k, v in t_stats.items():
                target_key = f"{mark}_tss_{prefix}_{k}" if k in base_keys[:5] else f"{mark}_tss_{k}"
                tss_features[target_key].append(v)


        bw.close()

        # === 合併 ===
        for feat_dict in (gene_features, tss_features):
            for col, vals in feat_dict.items():
                genes[col] = vals

    # === 儲存結果 ===
    out_suffix = {
        "none": "raw",
        "zscore": "zscore",
        "log_zscore": "logzscore",
        "log_only": "log"
    }[norm_mode]

    out_path = os.path.join(output_dir, f"{cell}_{out_suffix}.tsv")
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    genes.to_csv(out_path, sep="\t", index=False)

    print(f"\n✅ Saved {out_suffix} features for {cell} → {out_path}")


📖 Loaded reference: 18268 genes

🧬 Processing cell line: X1

📂 Reading DNase (X1) ...


  2%|▏         | 367/18268 [00:02<02:29, 120.03it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/4018982790.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/4018982790.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
100%|██████████| 18268/18268 [02:00<00:00, 151.62it/s]



📂 Reading H3K27ac (X1) ...


  0%|          | 45/18268 [00:00<02:36, 116.63it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/4018982790.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/4018982790.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
100%|██████████| 18268/18268 [02:01<00:00, 149.91it/s]



📂 Reading H3K4me3 (X1) ...


...


📂 Reading H3K4me1 (X3) ...


  1%|          | 165/18268 [00:01<01:48, 166.67it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/4018982790.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/4018982790.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
100%|██████████| 18268/18268 [02:07<00:00, 143.37it/s]
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/3177119086.py:126


📂 Reading H3K9me3 (X3) ...


  1%|          | 155/18268 [00:01<01:51, 161.98it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/4018982790.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  sharpness = kurtosis(vals_norm) if len(vals_norm) > 3 else 0.0
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/4018982790.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  asymmetry = skew(vals_norm) if len(vals_norm) > 3 else 0.0
/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
100%|██████████| 18268/18268 [02:04<00:00, 146.63it/s]
/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_55634/3177119086.py:126


✅ Saved logzscore features for X3 → /Users/deweywang/Desktop/GitHub/Gene-expression-prediction/preprocessed_data/reference/0. data/bigwig2000/X3_logzscore.tsv


In [23]:
import os
import pandas as pd
import glob
import re

# ============================================================
#                  基本設定
# ============================================================
base_dir = "../preprocessed_data/reference/0. raw_bigwig"
files = sorted(glob.glob(os.path.join(base_dir, "X*_zscore_dynamics.tsv")))

print(f"📂 Found {len(files)} files:")
for f in files:
    print(f"  - {os.path.basename(f)}")

# ============================================================
#                  欄位重新命名函式
# ============================================================
def rename_columns(df):
    rename_map = {}
    for col in df.columns:
        new_col = col
        new_col = re.sub(r"_z_mean$", "_mean", new_col)
        new_col = re.sub(r"_z_std$", "_std", new_col)
        new_col = re.sub(r"_z_min$", "_min", new_col)
        new_col = re.sub(r"_z_max$", "_max", new_col)
        new_col = re.sub(r"_z_diff$", "_diff", new_col)
        rename_map[col] = new_col
    return df.rename(columns=rename_map)

# ============================================================
#                  主流程
# ============================================================
mark_min_results = {}
summary_rows = []

for f in files:
    cell = os.path.basename(f).split("_")[0]
    print(f"\n🔧 Processing {cell} ...")

    df = pd.read_csv(f, sep="\t")
    df = rename_columns(df)

    # 儲存回原地
    df.to_csv(f, sep="\t", index=False)
    print(f"💾 Saved updated file: {f}")
    print(f"🧩 Columns: {len(df.columns)} total")
    
    # 檢查是否有任何 *_tss_min / *_gene_min 欄位
    tss_cols = [c for c in df.columns if "_tss_min" in c]
    gene_cols = [c for c in df.columns if "_gene_min" in c]

    if not tss_cols or not gene_cols:
        print(f"⚠️ No '_tss_min' or '_gene_min' columns found in {cell}. Skipping.")
        continue

    # === 找出每個 mark ===
    mark_cols = sorted({col.split("_")[0] for col in df.columns if "_tss_min" in col or "_gene_min" in col})
    print(f"✅ Marks found: {mark_cols}")

    for mark in mark_cols:
        tss_col = f"{mark}_tss_min"
        gene_col = f"{mark}_gene_min"

        if tss_col not in df.columns or gene_col not in df.columns:
            print(f"⚠️ Missing {tss_col} or {gene_col}")
            continue

        min_tss = df[tss_col].min(skipna=True)
        min_gene = df[gene_col].min(skipna=True)

        summary_rows.append({
            "mark": mark,
            "cell": cell,
            "tss_min": min_tss,
            "gene_min": min_gene
        })

# ============================================================
#                  結果輸出
# ============================================================
if summary_rows:
    summary_df = pd.DataFrame(summary_rows).sort_values(["mark", "cell"]).reset_index(drop=True)
    print("\n🧭 Summary of minimum values per mark:")
    print(summary_df)

    summary_path = os.path.join(base_dir, "summary_min_values.tsv")
    summary_df.to_csv(summary_path, sep="\t", index=False)
    print(f"\n✅ Summary saved to: {summary_path}")
else:
    print("\n❌ No marks found with '_tss_min' and '_gene_min' columns. Please check column names in your TSV files.")


📂 Found 3 files:
  - X1_zscore_dynamics.tsv
  - X2_zscore_dynamics.tsv
  - X3_zscore_dynamics.tsv

🔧 Processing X1 ...
💾 Saved updated file: ../preprocessed_data/reference/0. raw_bigwig/X1_zscore_dynamics.tsv
🧩 Columns: 175 total
✅ Marks found: ['DNase', 'H3K27ac', 'H3K27me3', 'H3K36me3', 'H3K4me1', 'H3K4me3', 'H3K9me3']

🔧 Processing X2 ...
💾 Saved updated file: ../preprocessed_data/reference/0. raw_bigwig/X2_zscore_dynamics.tsv
🧩 Columns: 175 total
✅ Marks found: ['DNase', 'H3K27ac', 'H3K27me3', 'H3K36me3', 'H3K4me1', 'H3K4me3', 'H3K9me3']

🔧 Processing X3 ...
💾 Saved updated file: ../preprocessed_data/reference/0. raw_bigwig/X3_zscore_dynamics.tsv
🧩 Columns: 175 total
✅ Marks found: ['DNase', 'H3K27ac', 'H3K27me3', 'H3K36me3', 'H3K4me1', 'H3K4me3', 'H3K9me3']

🧭 Summary of minimum values per mark:
        mark cell   tss_min  gene_min
0      DNase   X1  0.024102  0.024102
1      DNase   X2  0.003446  0.003446
2      DNase   X3  0.000000  0.000000
3    H3K27ac   X1  0.000000  0.00000